In [ ]:
import numpy as np
import pandas as pd
# import newspaper
# from newspaper import Article
from time import sleep
import requests
import json
import glob
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
bearer_token=''

In [ ]:
search_url = "https://api.twitter.com/2/tweets/search/all"

In [ ]:
fields='attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,public_metrics,possibly_sensitive,referenced_tweets,reply_settings,source,text,withheld'

In [ ]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers, params):
    response = requests.request("GET", search_url, headers=headers, params=params)
#     print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def main():
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(search_url, headers, query_params)
    #print(json.dumps(json_response, indent=4, sort_keys=True))
    return json_response

In [ ]:
# 
umb0='''mobility technology
inequality technology
equality technology
accessibility technology
migration technology
sustainability technology'''.split('\n')

umb1='''mobility 5g
inequality 5g
equality 5g
accessibility 5g
sustainability 5g'''.split('\n')

umb2='''mobility iot
inequality iot
equality iot
accessibility iot
migration iot
sustainability iot'''.split('\n')

umb3='''mobility metaverse
inequality metaverse
equality metaverse
accessibility metaverse
sustainability metaverse'''.split('\n')

umb3='''mobility ai
inequality ai
equality ai
accessibility ai
migration ai
sustainability ai'''.split('\n')

end_time='2022-06-01T00:00:00z'
start_time='2020-01-01T00:00:00z'
max_results='100'

In [ ]:
path='./umb3/'

for keyw in umb3:
    next_page=True
    query='{} lang:en -is:retweet -is:reply has:links -has:media'.format(keyw)
    df=pd.DataFrame()
    query_params = {'query': query,'tweet.fields':fields, 'end_time':end_time,'start_time':start_time, 'max_results':max_results}
    j=0
    while next_page is True:
        try:
            json_response=main()
        except:
            counter=0
            while counter<3:
                sleep(30)
                try: 
                    json_response=main()
                    counter=3
                except:
                    counter=counter+1
        df=df.append(pd.DataFrame(json_response['data']))
        print('df:'+str(len(df)))
        print(df['created_at'][-1:])
        if 'next_token' in json_response['meta'].keys() and json_response['meta']['result_count']!=0:
                    next_page=True
                    next_token=json_response['meta']['next_token']
                    sleep(2)
                    query_params = {'query': query,'tweet.fields':fields, 'end_time':end_time,'start_time':start_time, 'max_results':max_results,
                                    'next_token':next_token, 
                                    }
        else:
            next_page=False
        if len(df)>100000:
            df.to_csv2(path+query.split('lang')[0].strip()+'_'+str(j)+'.csv')
            j=j+1
            df=pd.DataFrame()
        if j==5:
            next_page=False

    df.to_csv(path+query.split('lang')[0].strip()+'_'+str(j)+'.csv')

all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0,lineterminator='\n')
    li.append(df)

dfm = pd.concat(li, axis=0, ignore_index=True)
dfm.to_csv(path+'all.csv')

df:99
98    2022-05-26T03:04:00.000Z
Name: created_at, dtype: object
df:199
99    2022-05-19T07:37:11.000Z
Name: created_at, dtype: object
df:299
99    2022-05-13T06:40:53.000Z
Name: created_at, dtype: object
df:399
99    2022-05-07T02:30:05.000Z
Name: created_at, dtype: object
df:499
99    2022-05-03T21:02:48.000Z
Name: created_at, dtype: object
df:599
99    2022-04-29T14:05:45.000Z
Name: created_at, dtype: object
df:699
99    2022-04-23T15:04:12.000Z
Name: created_at, dtype: object
df:799
99    2022-04-13T15:18:59.000Z
Name: created_at, dtype: object
df:898
98    2022-04-06T12:25:52.000Z
Name: created_at, dtype: object
df:997
98    2022-03-30T20:32:56.000Z
Name: created_at, dtype: object
df:1096
98    2022-03-23T18:08:34.000Z
Name: created_at, dtype: object
df:1196
99    2022-03-16T11:15:33.000Z
Name: created_at, dtype: object
df:1296
99    2022-03-08T21:55:29.000Z
Name: created_at, dtype: object
df:1396
99    2022-02-27T01:06:20.000Z
Name: created_at, dtype: object
df:1496
99    202

In [ ]:
all_files = glob.glob("/content/drive/My Drive/idub_mk/all/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0,lineterminator='\n')
    li.append(df)

dfm = pd.concat(li, axis=0, ignore_index=True)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
len(dfm)

256682

In [ ]:
len(dfm)

256682

In [ ]:
df=dfm

In [ ]:
def process(row):
    x=row['entities']
    try:
        y=eval(x)
        return y
    except:
        return 'zero' 

In [ ]:
df['entities']=df.apply(lambda row: process(row), axis=1)

In [ ]:
df=df[(df['entities'].apply(lambda x: 'urls' in str(x))==True)]

In [ ]:
len(df)

256681

In [ ]:
df['url']=df['entities'].apply(lambda x: x['urls'])
df['url']=df['url'].apply(lambda x: x[0]['expanded_url'])

<ipython-input-13-3286e20dfe81>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['url']=df['entities'].apply(lambda x: x['urls'])
<ipython-input-13-3286e20dfe81>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['url']=df['url'].apply(lambda x: x[0]['expanded_url'])


In [ ]:
len(df.drop_duplicates(subset=['url']))

146553

In [ ]:
df=df.drop_duplicates(subset=['url'])

In [ ]:
df['domain']=df['url'].str.extract('(?<=http://)(.*?)(?=/)|(?<=https://)(.*?)(?=/)')[1]
df['domain']=df['domain'].fillna(df['url'])

In [ ]:
df['domain'].value_counts()[:10]

bit.ly         14498
twitter.com    12774
paper.li        8943
lnkd.in         8178
ift.tt          3319
buff.ly         2272
t.eric.sn       2262
hubs.ly         1973
youtu.be        1397
stuf.in         1216
Name: domain, dtype: int64

In [ ]:
df_news=df[~df['domain'].isin(['lttr.ai','bit.ly','twitter.com','lnkd.in','paper.li','ift.tt','buff.ly','youtu.be','hubs.ly','stuf.in','www.youtube.com','zpr.io','tinyurl.com','t.her.is','hubs.li','t.eric.sn','hubs.la','trib.al','qoo.ly','www.linkedin.com'])]

In [ ]:
t=pd.DataFrame(df_news['domain'].value_counts()).reset_index()

In [ ]:
t[t['index'].str.contains('wired')]

,index,domain
84,www.wired.com,95
393,www.wired.co.uk,21
917,wired.trib.al,10
2367,wired.me,4
5256,www-wired-com.cdn.ampproject.org,2
19008,http://agwired.com/2021/07/07/excitement-growi...,1
22594,http://agwired.com/2020/01/09/promoting-sustai...,1
24456,http://dpmd.ai/wired21,1


In [ ]:
df_news[df_news['domain'].str.contains('wired')]['url']

1540      https://www.wired.com/story/why-tesla-designin...
1958      https://www.wired.com/story/use-self-driving-c...
3499      https://www.wired.com/story/blackberry-is-plan...
4027      https://www.wired.com/story/company-uses-ai-ou...
4633      https://www.wired.com/story/tesla-model-x-auto...
                                ...                        
243236    https://www.wired.co.uk/partnerships/wired-sma...
244825    https://www.wired.com/story/geoengineering-is-...
245179    https://www.wired.co.uk/article/nuclear-batteries
253879    https://www.wired.com/story/cheap-nanoparticle...
254184    https://www.wired.com/2015/09/design-issue-fut...
Name: url, Length: 135, dtype: object

In [ ]:
201+517+1119+827+96+421+174+149+93+135

3732

In [ ]:
1119/3732

0.29983922829581994

In [ ]:
96/3732

0.02572347266881029

In [ ]:
len(df_news[df_news['domain'].str.contains('technologyreview')])

201

In [ ]:
len(df_news[df_news['domain'].str.contains('guardian')])

517

In [ ]:
len(df_news[df_news['domain'].str.contains('forbes')])

1119

In [ ]:
len(df_news[df_news['domain'].str.contains('nyt')])

827

In [ ]:
len(df_news[df_news['domain'].str.contains('cnet')])

96

In [ ]:
len(df_news[df_news['domain'].str.contains('techcrunch')])

421

In [ ]:
len(df_news[df_news['domain'].str.contains('verge')])

174

In [ ]:
len(df_news[df_news['domain'].str.contains('venturebeat')])

149

In [ ]:
len(df_news[df_news['domain'].str.contains('zdnet')])

93

In [ ]:
len(df_news[df_news['domain'].str.contains('wired')])

135